# Restart Terminated Process Instance

This notebook provides an interactive tool to restart externally terminated process instances in Operaton/Camunda 7.

## Features
- Select a process definition from a dropdown
- View terminated instances with business key and end time
- Parse BPMN to show available activities for restart point selection
- Preview the process diagram with activity highlighting
- Restart the process from a selected activity

## How Process Restart Works
When you restart a terminated process instance:
1. A new process instance is created using the same process definition version
2. Historic variable values are restored
3. Execution starts from the specified activity
4. The new instance gets a new ID but references the original

## Usage
1. Run all cells in order
2. Select a process definition
3. Choose a terminated instance to restart
4. Select the activity to start from (or enter manually)
5. Preview the diagram and click "Restart Instance"

In [ ]:
# Imports and Environment Setup
import json
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import operaton
from operaton import Operaton

await operaton.load_env()
await operaton.load_bpmn_moddle()

print("✓ Environment loaded")

In [ ]:
# Fetch process definitions
definitions = Operaton.get('/process-definition?latestVersion=true')
definition_options = {f"{d['key']} (v{d['version']})": d['id'] for d in definitions}
print(f"✓ Found {len(definitions)} process definitions")

In [ ]:
# State management
state = {
    'bpmn_xml': None,
    'activities': {},  # {display_name: activity_id}
    'selected_instance': None,
}

# Helper to parse BPMN and extract activities
async def parse_activities_from_bpmn(xml):
    """Parse BPMN XML and extract activity IDs and names."""
    activities = {}
    try:
        result = await operaton.parse_bpmn(xml)
        root = result.rootElement
        
        # Get all processes
        for process in root.rootElements or []:
            if hasattr(process, '$type') and process['$type'] == 'bpmn:Process':
                for element in process.flowElements or []:
                    el_type = element.get('$type', '')
                    el_id = element.get('id', '')
                    el_name = element.get('name', '')
                    
                    # Include tasks, events, and gateways as potential restart points
                    if any(t in el_type for t in ['Task', 'Event', 'Gateway', 'SubProcess']):
                        display_name = f"{el_name} ({el_id})" if el_name else el_id
                        activities[display_name] = el_id
    except Exception as e:
        print(f"Warning: Could not parse BPMN: {e}")
    return activities

# Widgets
w_definition = widgets.Dropdown(
    options=[('-- Select --', '')] + list(definition_options.items()),
    description='Definition:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='500px')
)

w_instance = widgets.Dropdown(
    options=[('-- Select definition first --', '')],
    description='Instance:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='500px')
)

w_activity_dropdown = widgets.Dropdown(
    options=[('-- Select instance first --', '')],
    description='Activity:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='500px')
)

w_activity_manual = widgets.Text(
    value='',
    placeholder='Or enter activity ID manually',
    description='Manual ID:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='500px')
)

w_skip_custom_listeners = widgets.Checkbox(
    value=False,
    description='Skip custom listeners',
    layout=widgets.Layout(width='300px')
)

w_skip_io_mappings = widgets.Checkbox(
    value=False,
    description='Skip I/O mappings',
    layout=widgets.Layout(width='300px')
)

w_initial_variables = widgets.Checkbox(
    value=True,
    description='Use initial variables only (not latest)',
    layout=widgets.Layout(width='300px')
)

w_without_business_key = widgets.Checkbox(
    value=False,
    description='Without business key',
    layout=widgets.Layout(width='300px')
)

w_restart_btn = widgets.Button(
    description='Restart Instance',
    button_style='danger',
    icon='play',
    layout=widgets.Layout(width='200px')
)

w_preview_btn = widgets.Button(
    description='Preview Diagram',
    button_style='info',
    icon='eye',
    layout=widgets.Layout(width='200px')
)

out_instance_details = widgets.Output()
out_diagram = widgets.Output()
out_result = widgets.Output()

Dropdown(description='Process', options=((None, 'Process_17x20bq'), (None, 'Process_1c0pyl5'), (None, 'Process…

Output()

Dropdown(description='Business key', options=(), value=None)

Text(value='', description='Starting activity id')

Button(description='Execute', style=ButtonStyle())

Output()

In [ ]:
# Event Handlers
import asyncio

async def on_definition_change_async(definition_id):
    """Load terminated instances and parse BPMN for activities."""
    with out_result:
        clear_output()
    with out_instance_details:
        clear_output()
    with out_diagram:
        clear_output()
    
    if not definition_id:
        w_instance.options = [('-- Select definition first --', '')]
        w_activity_dropdown.options = [('-- Select instance first --', '')]
        return
    
    # Fetch BPMN XML and parse activities
    try:
        xml_response = Operaton.get(f'/process-definition/{definition_id}/xml')
        state['bpmn_xml'] = xml_response.get('bpmn20Xml', '')
        
        # Parse activities asynchronously
        state['activities'] = await parse_activities_from_bpmn(state['bpmn_xml'])
        
        if state['activities']:
            w_activity_dropdown.options = [('-- Select activity --', '')] + list(state['activities'].items())
        else:
            w_activity_dropdown.options = [('-- No activities found --', '')]
    except Exception as e:
        with out_result:
            print(f"Error loading BPMN: {e}")
    
    # Fetch terminated (externally terminated) instances
    try:
        instances = Operaton.get(
            f'/history/process-instance?processDefinitionId={definition_id}'
            f'&state=externallyTerminated&sortBy=endTime&sortOrder=desc&maxResults=100'
        )
        
        if instances:
            instance_options = [('-- Select --', '')]
            for inst in instances:
                bk = inst.get('businessKey', 'N/A')
                end_time = inst.get('endTime', 'N/A')[:19] if inst.get('endTime') else 'N/A'
                label = f"{inst['id'][:8]}... | BK: {bk} | Ended: {end_time}"
                instance_options.append((label, inst['id']))
            w_instance.options = instance_options
        else:
            w_instance.options = [('-- No terminated instances --', '')]
    except Exception as e:
        with out_result:
            print(f"Error loading instances: {e}")

def on_definition_change(change):
    if change['name'] == 'value':
        asyncio.create_task(on_definition_change_async(change['new']))

def on_instance_change(change):
    """Show instance details when selected."""
    if change['name'] != 'value' or not change['new']:
        return
    
    instance_id = change['new']
    state['selected_instance'] = instance_id
    
    with out_instance_details:
        clear_output(wait=True)
        try:
            instance = Operaton.get(f'/history/process-instance/{instance_id}')
            
            # Fetch historic activity instances
            activities = Operaton.get(
                f'/history/activity-instance?processInstanceId={instance_id}'
                f'&sortBy=startTime&sortOrder=asc'
            )
            
            html = f"""
            <h4>Instance Details</h4>
            <table style="border-collapse: collapse; width: 100%;">
                <tr><td style="padding: 4px; border: 1px solid #ddd;"><b>ID</b></td>
                    <td style="padding: 4px; border: 1px solid #ddd;">{instance.get('id', 'N/A')}</td></tr>
                <tr><td style="padding: 4px; border: 1px solid #ddd;"><b>Business Key</b></td>
                    <td style="padding: 4px; border: 1px solid #ddd;">{instance.get('businessKey', 'N/A')}</td></tr>
                <tr><td style="padding: 4px; border: 1px solid #ddd;"><b>Start Time</b></td>
                    <td style="padding: 4px; border: 1px solid #ddd;">{instance.get('startTime', 'N/A')}</td></tr>
                <tr><td style="padding: 4px; border: 1px solid #ddd;"><b>End Time</b></td>
                    <td style="padding: 4px; border: 1px solid #ddd;">{instance.get('endTime', 'N/A')}</td></tr>
                <tr><td style="padding: 4px; border: 1px solid #ddd;"><b>State</b></td>
                    <td style="padding: 4px; border: 1px solid #ddd;">{instance.get('state', 'N/A')}</td></tr>
                <tr><td style="padding: 4px; border: 1px solid #ddd;"><b>Delete Reason</b></td>
                    <td style="padding: 4px; border: 1px solid #ddd;">{instance.get('deleteReason', 'N/A')}</td></tr>
            </table>
            <h4>Activity History ({len(activities)} activities)</h4>
            <table style="border-collapse: collapse; width: 100%; font-size: 12px;">
                <tr style="background: #f0f0f0;">
                    <th style="padding: 4px; border: 1px solid #ddd;">Activity ID</th>
                    <th style="padding: 4px; border: 1px solid #ddd;">Name</th>
                    <th style="padding: 4px; border: 1px solid #ddd;">Type</th>
                    <th style="padding: 4px; border: 1px solid #ddd;">Start</th>
                    <th style="padding: 4px; border: 1px solid #ddd;">End</th>
                </tr>
            """
            for act in activities[:20]:  # Limit to 20
                html += f"""
                <tr>
                    <td style="padding: 4px; border: 1px solid #ddd;">{act.get('activityId', '')}</td>
                    <td style="padding: 4px; border: 1px solid #ddd;">{act.get('activityName', '') or '-'}</td>
                    <td style="padding: 4px; border: 1px solid #ddd;">{act.get('activityType', '')}</td>
                    <td style="padding: 4px; border: 1px solid #ddd;">{(act.get('startTime') or '')[:19]}</td>
                    <td style="padding: 4px; border: 1px solid #ddd;">{(act.get('endTime') or '')[:19]}</td>
                </tr>
                """
            html += "</table>"
            if len(activities) > 20:
                html += f"<p><i>... and {len(activities) - 20} more activities</i></p>"
            
            display(HTML(html))
        except Exception as e:
            print(f"Error loading instance details: {e}")

def on_preview_click(btn):
    """Render BPMN diagram with activity highlighting."""
    with out_diagram:
        clear_output(wait=True)
        
        if not state['bpmn_xml']:
            print("No BPMN loaded. Select a process definition first.")
            return
        
        # Get selected activity
        activity_id = w_activity_manual.value.strip() or w_activity_dropdown.value
        
        # Get historic activities for the selected instance
        executed_activities = []
        if state['selected_instance']:
            try:
                acts = Operaton.get(
                    f'/history/activity-instance?processInstanceId={state["selected_instance"]}'
                )
                executed_activities = list(set(a.get('activityId', '') for a in acts))
            except:
                pass
        
        # Build config for BPMN renderer
        config = {
            "style": {"height": "400px"},
            "zoom": "fit-viewport",
            "activities": executed_activities,
        }
        
        # Highlight selected restart activity in green
        if activity_id:
            config["colors"] = {
                activity_id: {"stroke": "#000000", "fill": "#52B415"}
            }
        
        display({
            "application/bpmn+xml": state['bpmn_xml'],
            "application/bpmn+json": json.dumps(config)
        }, raw=True)

def on_restart_click(btn):
    """Execute the process instance restart."""
    with out_result:
        clear_output(wait=True)
        
        instance_id = w_instance.value
        if not instance_id:
            print("⚠️ Please select an instance to restart.")
            return
        
        # Get activity ID from manual input or dropdown
        activity_id = w_activity_manual.value.strip() or w_activity_dropdown.value
        if not activity_id:
            print("⚠️ Please select or enter an activity ID.")
            return
        
        definition_id = w_definition.value
        
        # Build restart payload
        payload = {
            "processInstanceIds": [instance_id],
            "instructions": [{
                "type": "startBeforeActivity",
                "activityId": activity_id
            }],
            "skipCustomListeners": w_skip_custom_listeners.value,
            "skipIoMappings": w_skip_io_mappings.value,
            "initialVariables": w_initial_variables.value,
            "withoutBusinessKey": w_without_business_key.value
        }
        
        print(f"Restarting instance {instance_id}...")
        print(f"Activity: {activity_id}")
        print(f"Payload: {json.dumps(payload, indent=2)}")
        print()
        
        try:
            result = Operaton.post(
                f'/process-definition/{definition_id}/restart',
                json=payload
            )
            print("✅ Restart successful!")
            if result:
                print(f"Response: {json.dumps(result, indent=2)}")
        except Exception as e:
            print(f"❌ Restart failed: {e}")

# Attach handlers
w_definition.observe(on_definition_change, names='value')
w_instance.observe(on_instance_change, names='value')
w_preview_btn.on_click(on_preview_click)
w_restart_btn.on_click(on_restart_click)

In [ ]:
# Display UI
display(HTML("<h3>1. Select Process Definition</h3>"))
display(w_definition)

display(HTML("<h3>2. Select Terminated Instance</h3>"))
display(w_instance)
display(out_instance_details)

display(HTML("<h3>3. Select Restart Activity</h3>"))
display(widgets.VBox([
    w_activity_dropdown,
    w_activity_manual,
    widgets.HTML("<small><i>Select from parsed activities or enter ID manually</i></small>")
]))

display(HTML("<h3>4. Options</h3>"))
display(widgets.VBox([
    w_skip_custom_listeners,
    w_skip_io_mappings,
    w_initial_variables,
    w_without_business_key
]))

display(HTML("<h3>5. Preview & Restart</h3>"))
display(widgets.HBox([w_preview_btn, w_restart_btn]))
display(out_diagram)
display(out_result)